### 라이브러리 불러오기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob, gc

In [ ]:
pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',250)

### 공휴일 데이터 불러오기

In [ ]:
holiday = pd.read_csv('./subway/holiday.csv', encoding='cp949')
holiday['날짜1'] = pd.to_datetime(holiday['날짜1'])
holiday


### 지하철 데이터 불러오기 
- 불러온 데이터에서 역 호선을 1~8 으로 매핑

In [ ]:
# './subway/' 폴더 내의 '* 20* *.csv' 파일들을 모두 찾아서 리스트로 저장
all_subway = glob.glob(os.path.join('./subway/', '* 20* *.csv'))

# 호선 번호와 호선 이름을 매핑한 딕셔너리 생성
hosun = {f'{i}호선': i for i in range(1, 9)}
print(hosun)

# all_subway 리스트의 각 파일에 대해 반복문 실행
for idx, f in enumerate(all_subway):
    # 파일을 읽어서 DataFrame으로 저장
    subway = pd.read_csv(f, encoding='cp949', low_memory=False)
    
    # '00~01' 열의 결측값을 0으로 대체
    subway['00~01'].fillna(0, inplace=True)
    
    # 5번째 열부터 끝까지 각 열에 대해 데이터 타입 변환 작업 수행
    for c in subway.columns[5:]:
        try:
            # ',' 문자 제거 후 문자열을 숫자로 변환
            subway[c] = subway[c].str.replace(',', '')
            subway[c] = pd.to_numeric(subway[c], errors='ignore')
            subway[c] = subway[c].astype(np.float64)
        except Exception as e:
            # 변환 중 에러가 발생하면 그대로 유지
            subway[c] = pd.to_numeric(subway[c], errors='ignore')
            subway[c] = subway[c].astype(np.float64)
    
    # 첫 번째 파일인 경우 all_sub에 subway를 할당하고, 그 이후 파일은 이어붙임
    all_sub = subway if idx == 0 else pd.concat([all_sub, subway], axis=0)


In [ ]:
# '역번호' 열의 데이터 타입을 숫자로 변환하되, 변환할 수 없는 값은 NaN으로 처리
all_sub['역번호'] = pd.to_numeric(all_sub['역번호'], errors='coerce')

# '호선' 열의 결측값을 0으로 대체
all_sub['호선'].fillna(0, inplace=True)


In [ ]:
# '역번호' 열의 결측값을 뒤의 다음 유효한 값으로 채움 (뒤로 채우기)
all_sub['역번호'].fillna(method='bfill', inplace=True)

# '역번호' 열의 데이터 타입을 64비트 정수형으로 변환
all_sub['역번호'] = all_sub['역번호'].astype(np.int64)


In [ ]:
all_sub[all_sub['역번호'].isna()]

In [ ]:
# '역번호' 열의 고유한 값들을 가져와서 반복문 수행
st = {st_num: list(set(all_sub[all_sub['역번호'] == st_num]['역명']))[0].strip()
      for st_num in all_sub['역번호'].unique()}

# 결과로 생성된 딕셔너리 출력
st


In [ ]:
# 빈 딕셔너리 생성
st_line_dict = {}

# '역번호' 열의 고유한 값들을 가져와서 반복문 수행
for st_num in all_sub['역번호'].unique():
    # '역번호'가 특정 값인 행들을 필터링하여 '호선' 열의 고유한 값들을 가져옴
    st_line = list(set(all_sub[all_sub['역번호'] == st_num]['호선']))
    
    # '호선' 값 중에서 문자열이 아니고 0이 아닌 값들을 필터링하여 첫 번째 값을 선택
    st_line = [l for l in st_line if (isinstance(l, str) is False) and (l != 0)][0]
    
    # '역번호'를 키(key)로, 해당 '역번호'에 대응하는 '호선'을 값(value)으로 딕셔너리에 추가
    st_line_dict[st_num] = st_line


In [ ]:
# '역번호' 열의 값을 기준으로 st_line_dict 딕셔너리를 사용하여 '호선' 열의 값을 매핑
all_sub['호선'] = all_sub['역번호'].map(st_line_dict)


In [ ]:
all_sub

In [ ]:
# 2008~2023_data_ver1
all_sub.to_csv('./2008~2023_data_ver1.csv', encoding='cp949', index=False)

In [ ]:
def pivot_df(df_in, val_name):
    # 변환할 시간대 열들을 리스트로 정의
    time_columns = ['05~06', '06~07', '07~08', '08~09', '09~10', '10~11', '11~12', '12~13', '13~14',
                    '14~15', '15~16', '16~17', '17~18', '18~19', '19~20', '20~21', '21~22', '22~23', '23~24', '00~01']

    # melt 함수를 사용하여 데이터프레임을 변환
    df_out = pd.melt(df_in, id_vars=['날짜', '호선', '역번호', '역명', '구분'],
                     value_vars=time_columns, var_name='시간', value_name=val_name)

    # 열 이름 영어로 변경
    df_out.columns = ['Date', 'Line_num', 'Station_num', 'Station', 'Division', 'Time', val_name]

    # 'Date'와 'Time' 열을 기준으로 정렬
    df_out.sort_values(['Date', 'Time'], inplace=True)

    # 인덱스 재설정
    df_out.reset_index(drop=True, inplace=True)

    return df_out


def merge_holiday(left_df, right_df, left='Date', right='날짜1', how ='left'):
    
    df_new = pd.merge(left_df, right_df, left_on=left, right_on=right, how=how)
    
    # 공휴일 컬럼 만들기 공휴일일때 1 아닐때 0
    df_new['holiday'] = df_new['휴일명'].apply(lambda x: 1 if pd.notnull(x) else 0)

    # 필요없는 컬럼 삭제 
    df_new.drop(['날짜1','휴일명'], axis=1, inplace=True)
    
    df_new['Date'] = pd.to_datetime(df_new['Date'])

    # 월요일 : 1, 화요일 : 2, ..., 일요일 : 7
    df_new['weekday'] = df_new['Date'].dt.weekday + 1
    df_new['weekday'] = df_new['weekday'].replace(7,0)

    # 토요일, 일요일, 공휴일 일때 휴일 1로 지정 
    df_new['holiday'] = np.where((df_new['holiday'] == 1) | (df_new['weekday'].isin([0, 6])), 1, 0)
    df_new['holiday'].value_counts()
    return df_new

In [ ]:
# pivot_df 함수를 사용하여 all_sub 데이터프레임을 변환하여 time_mel_df에 저장
time_mel_df = pivot_df(df_in=all_sub, val_name='flow')

# 변환된 데이터프레임인 time_mel_df 출력
time_mel_df


In [ ]:
# Division 열의 문자열에서 좌우 공백을 제거(strip)
time_mel_df.Division = time_mel_df.Division.str.strip()

In [ ]:
# 'Date' 열의 데이터 타입을 datetime으로 변환
time_mel_df['Date'] = pd.to_datetime(time_mel_df['Date'])


In [ ]:
# merge_holiday 함수를 호출하여 time_mel_df와 holiday 데이터프레임을 병합하여 merged_df에 저장
merged_df = merge_holiday(left_df=time_mel_df, right_df=holiday)


In [ ]:
# merged_df에서 'Division'과 'flow' 열을 선택하여 pivot 함수를 사용하여 변환
flow_df = merged_df[['Division', 'flow']].pivot(columns='Division', values='flow')

In [ ]:
# 'Division'과 'flow' 열을 제외한 열들을 선택하여 info_df에 저장
info_df = merged_df[merged_df.columns.difference(['Division', 'flow'])]

# info_df 출력
info_df


In [ ]:
# flow_df의 첫 번째 열에 해당하는 값들을 선택하여 NaN이 아닌 행들로 이루어진 get_on_v 데이터프레임 생성
get_on_v = pd.DataFrame(flow_df.iloc[:, 0]).dropna()

# flow_df의 두 번째 열에 해당하는 값들을 선택하여 NaN이 아닌 행들로 이루어진 get_off_v 데이터프레임 생성
get_off_v = pd.DataFrame(flow_df.iloc[:, 1]).dropna()



In [ ]:
# get_on_v와 get_off_v를 reset_index를 사용하여 인덱스 재설정 후, 열 방향(axis=1)으로 병합하여 rec_flow 데이터프레임 생성
rec_flow = pd.concat([get_on_v.reset_index(drop=True), get_off_v.reset_index(drop=True)], axis=1)

# rec_flow의 열 이름을 'geton'과 'getoff'로 변경
rec_flow.columns = ['geton', 'getoff']

# 'geton'과 'getoff' 열을 더하여 'get_all' 열 생성
rec_flow['get_all'] = rec_flow['geton'] + rec_flow['getoff']

# info_df에서 get_on_v의 인덱스에 해당하는 행들을 선택하여 인덱스 재설정 후, rec_flow와 열 방향으로 병합하여 recon_df 데이터프레임 생성
recon_df = pd.concat([info_df.loc[get_on_v.index, :].reset_index(drop=True), rec_flow], axis=1)

# recon_df 출력
recon_df


In [ ]:
recon_df.isna().sum()

In [ ]:
recon_df.dropna(axis=0, inplace=True)

In [ ]:

time_map = {
    '05~06': '05:00',
    '06~07': '06:00',
    '07~08': '07:00',
    '08~09': '08:00',
    '09~10': '09:00',
    '10~11': '10:00',
    '11~12': '11:00',
    '12~13': '12:00',
    '13~14': '13:00',
    '14~15': '14:00',
    '15~16': '15:00',
    '16~17': '16:00',
    '17~18': '17:00',
    '18~19': '18:00',
    '19~20': '19:00',
    '20~21': '20:00',
    '21~22': '21:00',
    '22~23': '22:00',
    '23~24': '23:00',
    '00~01': '00:00'
}

In [ ]:
# 'Time' 열의 값을 time_map을 사용하여 매핑(mapping)
recon_df['Time'] = recon_df['Time'].map(time_map)

In [ ]:
recon_df

In [ ]:
weather = pd.read_csv('./weather/2008~2023_weather.csv', encoding='cp949')
weather.shape

In [ ]:
# weather 데이터프레임의 '강수량(mm)'와 '적설(cm)' 열의 NaN 값을 0으로 대체하여 채우기
weather[['강수량(mm)', '적설(cm)']] = weather[['강수량(mm)', '적설(cm)']].fillna(0)

# 변경된 weather 데이터프레임 출력
weather


In [ ]:
# '일시' 열의 데이터 타입을 datetime으로 변환
weather['일시'] = pd.to_datetime(weather['일시'])

# '일시' 열에서 날짜를 추출하여 'Date' 열 생성
weather['Date'] = weather['일시'].dt.strftime('%Y-%m-%d')

# '일시' 열에서 시간을 추출하여 'Time' 열 생성
weather['Time'] = weather['일시'].dt.strftime('%H:%M')

# 필요없는 열들을 삭제하고 열 이름 변경
weather = weather.drop(columns=['지점', '지점명', '일시'])
weather.columns = ['Temp', 'Rainfall_amt', 'Wind_speed', 'Humidity', 'Snow_amt', 'Date', 'Time']

# 변경된 weather 데이터프레임 출력
weather


In [ ]:
# 날짜별로 평균을 계산하여 w_mean 데이터프레임 생성
w_mean = weather.groupby(['Date']).mean()

# 열 이름에 '_mean'을 추가하여 열 이름 변경
w_mean.columns = [f'{c}_mean' for c in w_mean.columns]

# 날짜별로 강수량과 적설량을 합산하여 w_sum 데이터프레임 생성
w_sum = weather.groupby(['Date'])[['Rainfall_amt', 'Snow_amt']].sum()

# 열 이름에 '_sum'을 추가하여 열 이름 변경
w_sum.columns = [f'{c}_sum' for c in w_sum.columns]

# 날짜별로 최고 기온을 계산하여 w_max 데이터프레임 생성
w_max = weather.groupby(['Date'])[['Temp']].max()

# 열 이름에 '_max'를 추가하여 열 이름 변경
w_max.columns = [f'{c}_max' for c in w_max.columns]

# 날짜별로 최저 기온을 계산하여 w_min 데이터프레임 생성
w_min = weather.groupby(['Date'])[['Temp']].min()

# 열 이름에 '_min'을 추가하여 열 이름 변경
w_min.columns = [f'{c}_min' for c in w_min.columns]

# w_mean, w_sum, w_max, w_min을 열 방향으로 병합하여 w_stats 데이터프레임 생성
w_stats = pd.concat([w_mean, w_sum, w_max, w_min], axis=1)

# 최고 기온과 최저 기온의 차이를 계산하여 'Temp_diff' 열 생성
w_stats['Temp_diff'] = np.abs(w_max.values - w_min.values)

# w_stats를 데이터프레임으로 변환하고 인덱스를 재설정
w_stats = pd.DataFrame(w_stats).reset_index(drop=False)

# w_stats 출력
display(w_stats)


In [ ]:
# weather와 w_stats를 'Date' 열을 기준으로 left 조인하여 weather 데이터프레임에 추가
weather = pd.merge(left=weather, right=w_stats, how='left', on='Date')

# 변경된 weather 데이터프레임 출력
weather


In [ ]:
def merge_weather(sub_df, weather_df):
    """
    지하철 데이터와 날씨 데이터를 병합하는 함수입니다.
    
    Args:
        sub_df (DataFrame): 지하철 데이터프레임.
        weather_df (DataFrame): 날씨 데이터프레임.
        
    Returns:
        DataFrame: 병합된 지하철-날씨 데이터프레임.
    """
    # 날짜 열을 datetime 형식으로 변환
    sub_df['Date'] = pd.to_datetime(sub_df['Date'])
    weather_df['Date'] = pd.to_datetime(weather_df['Date'])
    
    # 지하철과 날씨 데이터를 날짜와 시간을 기준으로 left 조인하여 병합
    subway_weather_df = pd.merge(sub_df, weather_df, on=['Date', 'Time'], how='left')
    
    # 날짜와 시간을 기준으로 정렬
    subway_weather_df.sort_values(['Date', 'Time'], inplace=True)
    subway_weather_df.reset_index(drop=True, inplace=True)
    
    # 시간, 연도, 월, 일을 추출하여 열로 추가
    subway_weather_df['hour'] = pd.to_datetime(subway_weather_df['Time']).dt.hour
    subway_weather_df["year"] = subway_weather_df["Date"].dt.year
    subway_weather_df["month"] = subway_weather_df["Date"].dt.month
    subway_weather_df["day"] = subway_weather_df["Date"].dt.day
    
    return subway_weather_df



In [ ]:
# recon_df와 weather를 이용하여 merge_weather 함수를 호출하여 dfnew_all에 저장
dfnew_all = merge_weather(sub_df=recon_df, weather_df=weather)


In [ ]:
dfnew_all

In [ ]:
dfnew_all.isnull().sum()

In [ ]:
dfnew_all = dfnew_all.drop(dfnew_all[dfnew_all['get_all'] == 0].index)


In [ ]:
dfnew_all.dropna(axis=0, inplace=True)

In [ ]:
dfnew_all.to_csv('./2008~2023_data.csv',encoding='cp949', index=False)

In [ ]:
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path, encoding='cp949')
    df.to_parquet(f'./{save_name}.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

In [ ]:
csv_to_parquet('./2008~2023_data.csv', '2008~2023_data')